## Valmistautuminen

Jos käytössäsi on Windows-kone, saatat joutua tekemään lisäasennuksia jotta saat [fastText-kirjaston asennettua](https://medium.com/@oleg.tarasov/building-fasttext-python-wrapper-from-source-under-windows-68e693a68cbb). 
Toinen vaihtoehto on toteuttaa tämä harjoitus [Google Colabissa](https://colab.research.google.com/). 


1. Asennettava: <code>fasttext</code>, <code>nltk</code>, <code>spacy</code> 
1. Käynnistä kernel uudelleen
1. Jos virheet jatkuvat, asenna:  <code>spacy-transformers</code> 
1. Käynnistä kernel uudelleen
1. <code>python -m spacy download fi_core_news_sm</code>



Following Teemu's presentation earlier this week, the trained FastText classifier was downloaded using the following lines:
<code>
    ft_classifier.quantize(input=input, retrain=True)
    ft_classifier.save_model("ft_classifier.ftz")
</code>

In [1]:
import fasttext
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
# import libvoikko #perusmutoistamiseen käytetty kirjasto
import spacy

In [64]:
# !python -m spacy download fi_core_news_sm

In [2]:
nlp = spacy.load("fi_core_news_sm")

In [4]:

print(stopwords.words('finnish'))

['olla', 'olen', 'olet', 'on', 'olemme', 'olette', 'ovat', 'ole', 'oli', 'olisi', 'olisit', 'olisin', 'olisimme', 'olisitte', 'olisivat', 'olit', 'olin', 'olimme', 'olitte', 'olivat', 'ollut', 'olleet', 'en', 'et', 'ei', 'emme', 'ette', 'eivät', 'minä', 'minun', 'minut', 'minua', 'minussa', 'minusta', 'minuun', 'minulla', 'minulta', 'minulle', 'sinä', 'sinun', 'sinut', 'sinua', 'sinussa', 'sinusta', 'sinuun', 'sinulla', 'sinulta', 'sinulle', 'hän', 'hänen', 'hänet', 'häntä', 'hänessä', 'hänestä', 'häneen', 'hänellä', 'häneltä', 'hänelle', 'me', 'meidän', 'meidät', 'meitä', 'meissä', 'meistä', 'meihin', 'meillä', 'meiltä', 'meille', 'te', 'teidän', 'teidät', 'teitä', 'teissä', 'teistä', 'teihin', 'teillä', 'teiltä', 'teille', 'he', 'heidän', 'heidät', 'heitä', 'heissä', 'heistä', 'heihin', 'heillä', 'heiltä', 'heille', 'tämä', 'tämän', 'tätä', 'tässä', 'tästä', 'tähän', 'tallä', 'tältä', 'tälle', 'tänä', 'täksi', 'tuo', 'tuon', 'tuotä', 'tuossa', 'tuosta', 'tuohon', 'tuolla', 'tuolta', 

In [5]:
# Load tweets datasets
# Note: May need to install openpyxl library for the following line to work
# pip install openpyxl
df = pd.read_excel('shared_data/sample_texts.xlsx')


In [65]:
df.head()

,Unnamed: 0,text_nousernames,author_username,Name,Position,eduskunta-vkk,twitter_handle
0,0,RT : Credit where due. Biden warned of the Ru...,TyttiTup,Tytti Tuppurainen,Eurooppa- ja omistajaohjausministeri,"eurooppa-, kulttuuri- ja urheiluministeri",TyttiTup
1,1,RT : I look forward to welcoming Sweden and F...,TyttiTup,Tytti Tuppurainen,Eurooppa- ja omistajaohjausministeri,"eurooppa-, kulttuuri- ja urheiluministeri",TyttiTup
2,2,"RT : ""Hätten wir damals wegen der Krim stärke...",TyttiTup,Tytti Tuppurainen,Eurooppa- ja omistajaohjausministeri,"eurooppa-, kulttuuri- ja urheiluministeri",TyttiTup
3,3,RT : My deepest condolences for the victims a...,TyttiTup,Tytti Tuppurainen,Eurooppa- ja omistajaohjausministeri,"eurooppa-, kulttuuri- ja urheiluministeri",TyttiTup
4,4,RT : Thank you Kamala Harris and \n for an...,TyttiTup,Tytti Tuppurainen,Eurooppa- ja omistajaohjausministeri,"eurooppa-, kulttuuri- ja urheiluministeri",TyttiTup


In [7]:
tweets = df[['text_nousernames', 'eduskunta-vkk']]

In [66]:
tweets.head()

,text_nousernames,eduskunta-vkk,label,preprocessed
0,RT : Credit where due. Biden warned of the Ru...,"eurooppa-, kulttuuri- ja urheiluministeri","__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt credit where due.biti warned of the russ...
1,RT : I look forward to welcoming Sweden and F...,"eurooppa-, kulttuuri- ja urheiluministeri","__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt look forward to welcoming sweden and fi...
2,"RT : ""Hätten wir damals wegen der Krim stärke...","eurooppa-, kulttuuri- ja urheiluministeri","__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt hätten wir damals wegen der krim stärke...
3,RT : My deepest condolences for the victims a...,"eurooppa-, kulttuuri- ja urheiluministeri","__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt my deepest condolences for the victims a...
4,RT : Thank you Kamala Harris and \n for an...,"eurooppa-, kulttuuri- ja urheiluministeri","__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt thank you kamala harris and for an...


In [9]:
# Methods imported from the original Notebook
def preprocess_word(word, stopwords):
    word = re.sub("[^A-Za-z0-9ÄäÖö-]+", "", word)
    word = re.sub("-", " ", word)
    word = word.strip().lower()
    if len(word) < 2:
        return ""
    if type(word) != None:
        if word in stopwords:
            return ""
        return word

def preprocess_sent(sent, stopwords):
    if sent != "":
        # lemmatize words using spacy
        doc = nlp(sent)
        sent = " ".join([token.lemma_ for token in doc])
        
        words = sent.strip().split(" ")
        final_sent = " ".join([preprocess_word(word, stopwords) for word in words if type(word) != None])
        return final_sent

def preprocess_row(teksti, stopwords):
    if type(teksti) == str:
        sents = teksti.strip().split(".")
        sents = filter(None,sents)
        teksti = ".".join([preprocess_sent(sent, stopwords) for sent in sents if type(sent) != None])
        return teksti

In [67]:
tweets.head()

,text_nousernames,eduskunta-vkk,label,preprocessed
0,RT : Credit where due. Biden warned of the Ru...,"eurooppa-, kulttuuri- ja urheiluministeri","__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt credit where due.biti warned of the russ...
1,RT : I look forward to welcoming Sweden and F...,"eurooppa-, kulttuuri- ja urheiluministeri","__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt look forward to welcoming sweden and fi...
2,"RT : ""Hätten wir damals wegen der Krim stärke...","eurooppa-, kulttuuri- ja urheiluministeri","__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt hätten wir damals wegen der krim stärke...
3,RT : My deepest condolences for the victims a...,"eurooppa-, kulttuuri- ja urheiluministeri","__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt my deepest condolences for the victims a...
4,RT : Thank you Kamala Harris and \n for an...,"eurooppa-, kulttuuri- ja urheiluministeri","__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt thank you kamala harris and for an...


In [11]:
#Apufunktio, jolla voidaan luoda fastText-yhteensopiva opetus- ja tekstitiedosto

def preprocess_ft_df(df):
    df["label"] = ["__label__" + "_".join(str(x).split(" ")) for x in df["eduskunta-vkk"].values]
    df["preprocessed"] = [preprocess_row(sentence, stopwords.words('finnish')) for sentence in df["text_nousernames"]]
    df = df.drop(["eduskunta-vkk", "text_nousernames"], axis = 1)
    return df

In [12]:
new_df = preprocess_ft_df(tweets)

/var/folders/9y/2jq2trpj7l564lf9sd493lbd7r_6wb/T/ipykernel_19314/2270654315.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = ["__label__" + "_".join(str(x).split(" ")) for x in df["eduskunta-vkk"].values]
/var/folders/9y/2jq2trpj7l564lf9sd493lbd7r_6wb/T/ipykernel_19314/2270654315.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["preprocessed"] = [preprocess_row(sentence, stopwords.words('finnish')) for sentence in df["text_nousernames"]]


In [68]:
new_df.head()

,label,preprocessed
0,"__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt credit where due.biti warned of the russ...
1,"__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt look forward to welcoming sweden and fi...
2,"__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt hätten wir damals wegen der krim stärke...
3,"__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt my deepest condolences for the victims a...
4,"__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt thank you kamala harris and for an...


In [14]:

model = fasttext.load_model("model_filename.ftz")

In [69]:
# TODO: Produce predictions

predictions = list()

for row in new_df.itertuples():
    # model.predict(row.preprocessed)[0][0]
    prediction = model.predict(row.preprocessed)[0][0] 
    # print(prediction)
    # print(predictions(model.predict(row.preprocessed)

    predictions.append(prediction)

# predictions


In [70]:
model.predict('Hiilen sidonta vaatii toimenpiteitä metsäpolitiikassa')[0][0]


'__label__maatalous-_ja_ympäristöministeri'

In [53]:
hits = 0
miss = 0
for pred, truth in zip(predictions, new_df['label'].values):
    if pred==truth:
        hits += 1
    else:
        miss += 1

In [54]:
hits, miss

(180, 988)

In [55]:
hits/miss

0.18218623481781376

In [ ]:
len(new_df['label'].unique())

16